In [ ]:
from Bio import SeqIO
import re
import csv
import duckdb
from protein_db import ProteinDB
pdb = ProteinDB()


In [ ]:
import csv

#
# 2,588,170 lines in nodes.dump - took 27mins to load
#
nodes_file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/taxonomy/nodes.dmp"

con = duckdb.connect(database=ProteinDB.db_string)
#con.execute("DROP TABLE TAX_NAME")
con.execute(" \
    CREATE TABLE TAX_NODE (\
                TAX_ID VARCHAR,\
                PARENT_TAX_ID VARCHAR,\
                EMBL_CODE VARCHAR)" \
                )

with open(nodes_file, mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader:
        #print(f'{row[0]} : {row[1]} : {row[2]}')
        id = row[0]
        parent = row[1]
        code = row[2]
        
        con.execute("INSERT INTO TAX_NODE (TAX_ID, PARENT_TAX_ID, EMBL_CODE) VALUES(?,?,?)", (id, parent, code))
        
        line_count += 1
        if (line_count % 10000 == 0):
            print(f'Processed {line_count} lines.')
con.close()

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)
con.execute("CREATE INDEX TN_TXIDX ON TAX_NODE(TAX_ID)")
con.close()

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)
nodes = con.execute("SELECT * FROM TAX_NODE").fetchall()
for i in range(10):
    print(nodes[i])
con.close()


In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)

#specific_id = '2759' # eukaryotic
specific_id = '2182604' # eukaryotic
#specific_id = '1872626'

print('Search for:', specific_id)

cat_species_result = con.execute("SELECT SPECIES_ID, CAT_ID FROM TAX_CAT WHERE SPECIES_ID = ?", [specific_id] ).fetchall()
cat_cat_result = con.execute("SELECT SPECIES_ID, CAT_ID, TYPE FROM TAX_CAT WHERE CAT_ID = ?", [specific_id] ).fetchall()
name_result = con.execute("SELECT NAME, UNIQUE_NAME, NAME_CLASS FROM TAX_NAME WHERE TAX_ID = ? AND UNIQUE_NAME='scientific name'", [specific_id] ).fetchall()
node_name = '\t'+specific_id+'\t'
node_result = con.execute("SELECT TAX_ID, PARENT_TAX_ID, EMBL_CODE FROM TAX_NODE WHERE PARENT_TAX_ID = ?", [node_name]).fetchall()

print('cat by species:\t',cat_species_result)
print('cat by cat:\t',cat_cat_result)
print('name by cat:\t',name_result)
print('node by parent:\t',node_result)
con.close()

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)

species_result = con.execute("SELECT DISTINCT SPECIES_ID FROM TAX_CAT WHERE TYPE='E' ORDER BY SPECIES_ID" ).fetchall()
cat_result = con.execute("SELECT DISTINCT CAT_ID FROM TAX_CAT WHERE TYPE='E' ORDER BY CAT_ID").fetchall()

print('species:')
for i in range(10):
    species_id = species_result[i]
    species_name_result = con.execute("SELECT NAME FROM TAX_NAME WHERE TAX_ID = ?", species_id).fetchall()
    print(species_id, len(species_name_result), ':', species_name_result)

print('\n')

print('categories:')    
for i in range(10):
    cat_id = cat_result[i]
    cat_name_result = con.execute("SELECT NAME FROM TAX_NAME WHERE TAX_ID = ?", cat_id).fetchall()
    print(cat_result[i], ':', len(cat_name_result), ':', cat_name_result)

